*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [1]:
# import libraries
# try:
#   # %tensorflow_version only exists in Colab.
#   !pip install tf-nightly
# except Exception:
#   pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
# !pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.6.0


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2021-10-04 18:27:12--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.3’

train-data.tsv.3    100%[===================>] 349.84K  --.-KB/s    in 0.04s   

2021-10-04 18:27:12 (8.98 MB/s) - ‘train-data.tsv.3’ saved [358233/358233]

--2021-10-04 18:27:12--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.3’

valid-data.tsv.3    100%[========

In [3]:
train_data = pd.read_csv(
    train_file_path,
    encoding = 'latin-1',
    sep='\t',
    header=0,
    names=['label', 'message'],
    usecols=['label', 'message'],
    dtype={'label': 'str', 'message': 'str'})

print(train_data.shape)
train_data.tail()

(4178, 2)


,label,message
4173,ham,just woke up. yeesh its late. but i didn't fal...
4174,ham,what do u reckon as need 2 arrange transport i...
4175,spam,free entry into our Â£250 weekly competition j...
4176,spam,-pls stop bootydelious (32/f) is inviting you ...
4177,ham,tell my bad character which u dnt lik in me. ...


In [4]:
valid_data = pd.read_csv(
    test_file_path, 
    encoding = 'latin-1',
    sep='\t',
    header=0,
    names=['label', 'message'],
    usecols=['label', 'message'],
    dtype={'label': 'str', 'message': 'str'})

valid_data.shape

(1391, 2)

In [5]:
train_data.groupby('label').describe()

message                                                               
        count unique                                                top freq
label                                                                       
ham      3618   3429                             sorry, i'll call later   22
spam      560    505  hmv bonus special 500 pounds of genuine hmv vo...    3

In [6]:
# convert labels to numerical variables
train_data['label_num'] = train_data.label.map({'ham':0, 'spam':1})
valid_data['label_num'] = valid_data.label.map({'ham':0, 'spam':1})
train_data.head()

,label,message,label_num
0,ham,you can never do nothing,0
1,ham,"now u sound like manky scouse boy steve,like! ...",0
2,ham,mum say we wan to go then go... then she can s...,0
3,ham,never y lei... i v lazy... got wat? dat day Ã¼...,0
4,ham,in xam hall boy asked girl tell me the startin...,0


In [7]:
# text processing
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

def text_process(mess):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns a list of the cleaned text
    """
    STOPWORDS = stopwords.words('english') + ['u', 'ü', 'ur', '4', '2', 'im', 'dont', 'doin', 'ure']
    
    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return ' '.join([word for word in nopunc.split() if word.lower() not in STOPWORDS])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# tokenization: convert text strings to a list of tokens (words we want)
train_data['clean_msg'] = train_data.message.apply(text_process)
print(train_data.shape)
train_data.head()

(4178, 4)


,label,message,label_num,clean_msg
0,ham,you can never do nothing,0,never nothing
1,ham,"now u sound like manky scouse boy steve,like! ...",0,sound like manky scouse boy stevelike travelli...
2,ham,mum say we wan to go then go... then she can s...,0,mum say wan go go shun bian watch da glass exh...
3,ham,never y lei... i v lazy... got wat? dat day Ã¼...,0,never lei v lazy got wat dat day Ã¼ send da ur...
4,ham,in xam hall boy asked girl tell me the startin...,0,xam hall boy asked girl tell starting term dis...


In [9]:
valid_data['clean_msg'] = valid_data.message.apply(text_process)
print(valid_data.shape)

(1391, 4)


In [10]:
# prepare data for vectorization
X_train = train_data['clean_msg']
X_valid = valid_data['clean_msg']
y_train = train_data['label_num']
y_valid = valid_data['label_num']
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)
X_train.head()

(4178,)
(1391,)
(4178,)
(1391,)


0                                        never nothing
1    sound like manky scouse boy stevelike travelli...
2    mum say wan go go shun bian watch da glass exh...
3    never lei v lazy got wat dat day Ã¼ send da ur...
4    xam hall boy asked girl tell starting term dis...
Name: clean_msg, dtype: object

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

# instantiate the vectorizer
vect = CountVectorizer()
vect.fit(X_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [12]:
# learn training data vocabulary, then use it to create a document-term matrix
# combine fit and transform into a single step
X_train_dtm = vect.fit_transform(X_train)

In [13]:
# examine the document-term matrix
X_train_dtm

<4178x7982 sparse matrix of type '<class 'numpy.int64'>'
	with 35003 stored elements in Compressed Sparse Row format>

In [14]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_valid_dtm = vect.transform(X_valid)
X_valid_dtm

<1391x7982 sparse matrix of type '<class 'numpy.int64'>'
	with 9984 stored elements in Compressed Sparse Row format>

In [15]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
tfidf_transformer.fit(X_train_dtm)
tfidf_transformer.transform(X_train_dtm)

<4178x7982 sparse matrix of type '<class 'numpy.float64'>'
	with 35003 stored elements in Compressed Sparse Row format>

In [16]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [17]:
# train the model using X_train_dtm
%time nb.fit(X_train_dtm, y_train)

CPU times: user 3.93 ms, sys: 0 ns, total: 3.93 ms
Wall time: 4.33 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [18]:
# make class predictions for X_valid_dtm
y_pred_class = nb.predict(X_valid_dtm)

In [19]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_valid, y_pred_class)

0.9877785765636233

# FCC Challenge
You should create a function called predict_message that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

In [20]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  txt = [pred_text]
  prediction = []
  dtm = vect.transform(txt)
  pred = nb.predict(dtm)
  p = pred[0]
  label = {0: 'ham', 1: 'spam'}
  prediction.append(p)
  prediction.append(label[p])

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[0, 'ham']


In [21]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
